# Parsing and Cleaning PHEME RNR Dataset Events

This notebook cleans tweet level data generated from `lib/pheme_parsing.py` and aggregates this tabular, individual data to tabular thread-level data. It also provides a useful sanity check after making modifications to `lib/pheme_parsing.py`. 

## Instructions
1. Update the variable `event` in the cell below with one of the following events:
    1. germanwings-crash
    1. ferguson
    1. ottawashooting
    1. sydneysiege
    1. charliehebdo
1. Run all the cells in this notebook to generate thread-level CSV files in the `data/threads` directory.

In [57]:
# Load dependencies for this Jupyter Notebook
import pandas as pd
import numpy as np
import networkx as nx
from functools import reduce
from lib.util import fetch_tweets

event = "charliehebdo"  # Change this value to clear different PHEME datasets

## Parsing and Cleaning Data
This step takes the raw PHEME rumor dataset and saves it tabular format as CSV file. The original PHEME dataset consists of JSON files organized into directories by event and category (rumor or non-rumor). These three functions below parse the data, save it as a CSV file (if necessary), and load it into this notebook as a Pandas DataFrame from the "cached" CSV file.

In [58]:
gw = fetch_tweets(event)

##  Tweet Level Features

| Name/Column       | Description                   | Type   | Notes  |
|-------------------|-------------------------------|--------| ------ |
| is_rumor          | Was this classified as rumor  | "bool" (`int`) | *Classification done by journalists* |
| thread            | Source tweet id               | `str`  |                                                   |
| in_reply_tweet    | Tweet ID in reply to          | `str`  |                                                   |
| event             | Name of the PHEME event       | `str`  | Corresponds to event in the PHEME dataset         |
| tweet_id          | Unique ID for tweet           | `str`  | This field is the ID referenced in `in_reply_tweet`     |
| is_source_tweet   | Was this classified as rumor  | "bool" (`int`) |                                                   |
| in_reply_user     | User ID in reply to           | `str`  |                                                   |
| user_id           | Twitter User's ID             | `str`  | This field is the ID referenced in `in_reply_user` |
| tweet_length      | Number of characters in tweet | `int`  |                                                   |
| urls_count        | Number of URLS in tweet       | `int`  |                                                   |
| hashtags_count    | Number of hashtags in tweet   | `int`  |                                                   |
| retweet_count     | Times the tweet was retweeted | `int`  |                                                   |
| favorite_count    | Number of times favorited     | `int`  |                                                   |
| mentions_count    | Number of users mentioned     | `int`  |                                                   |
| is_truncated      | Is this tweet truncated       | "bool" (`int`) | Did User type > 140 characters. [See Tweet updates](https://developer.twitter.com/en/docs/tweets/tweet-updates) |
| created              | Datetime Tweet was created    | `datetime` | |
| has_smile_emoji      | Does Tweet contain ""?        | "bool" (`int`) | 😊 is the smile emoji |
| user.tweets_count    | User's tweet total, currently | `int`  | |
| user.verified        | Is Twitter user verified?     | "bool" (`int`) |                                                   |
| user.followers_count | Total number of followers  | `int` | |
| user.listed_count    | ?? | `int` | | 
| user.friends_count   | ?? | `int` | |
| user.time_zone       | Timezone of the user's Twitter account | `str` | |
| user.desc_length     | Length of the user's biographic description | `int` |
| user.has_bg_img      | Does user have a profile background image?  | "bool" (`int`) |
| user.default_pric    | Does the user have the default profile picture | "bool" (`int`) |
| user.created_at      | Date and time Twitter account was activated | `datetime` | |
| user.utc_dist        | TK | `int` | See [this blog post time and the Twitter API](https://zacharyst.com/2017/04/05/assigning-the-correct-time-to-a-twee) |

## Germanwings Crash

In [59]:
gw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38268 entries, 0 to 38267
Data columns (total 59 columns):
is_rumor                38268 non-null int64
thread                  38268 non-null object
in_reply_tweet          38268 non-null object
event                   38268 non-null object
tweet_id                38268 non-null object
is_source_tweet         38268 non-null int64
in_reply_user           38268 non-null object
user_id                 38268 non-null object
tweet_length            38268 non-null int64
symbol_count            38268 non-null int64
user_mentions           38268 non-null int64
urls_count              38268 non-null int64
media_count             38268 non-null int64
hashtags_count          38268 non-null int64
retweet_count           38268 non-null int64
favorite_count          38268 non-null int64
mentions_count          38268 non-null int64
is_truncated            38268 non-null int64
created                 38268 non-null float64
has_smile_emoji         3826

The `.head` method prints out the 5 first rows in the dataframe

In [60]:
gw.head()

,is_rumor,thread,in_reply_tweet,event,tweet_id,is_source_tweet,in_reply_user,user_id,tweet_length,symbol_count,...,sentimentscore,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,has_url_in_text
0,0,553535765997969408,nan,charliehebdo,553535765997969408,1,nan,1379288282,144,0,...,0.000,8,2,1,1,0,0,2,0,1
1,0,553535765997969408,5.535357659979694e+17,charliehebdo,553536824673861633,0,1379288282.0,628636580,135,0,...,-0.050,8,4,1,1,0,0,3,0,0
2,0,553535765997969408,5.535357659979694e+17,charliehebdo,553545896739086336,0,1379288282.0,514523937,128,0,...,0.000,10,2,0,1,0,0,3,0,0
3,0,553535765997969408,5.535357659979694e+17,charliehebdo,553536468782571520,0,1379288282.0,1623557642,122,0,...,-0.125,6,4,1,3,1,0,2,0,0
4,0,553535765997969408,5.535357659979694e+17,charliehebdo,553540960718962690,0,1379288282.0,61921063,105,0,...,0.500,4,2,1,2,1,1,2,0,1


## Boolean Columns

The `describe` method will give summary information about each column in the dataframe. Each of these columns, except `is_truncated` should have two unique values.

Just for a sanity check. The cell below converts these boolean columns into value of type `bool` and describes them.

In [61]:
bool_columns = ["is_rumor", "is_source_tweet", "is_truncated", 
                "has_smile_emoji", "user.verified", "user.has_bg_img", 
                "user.default_pic", "sensitive", "has_place", "has_coords", "user.notifications"]

gw[bool_columns].astype(bool).describe(include="bool")

,is_rumor,is_source_tweet,is_truncated,has_smile_emoji,user.verified,user.has_bg_img,user.default_pic,sensitive,has_place,has_coords,user.notifications
count,38268,38268,38268,38268,38268,38268,38268,38268,38268,38268,38268
unique,2,2,1,2,2,2,2,2,2,2,1
top,False,False,False,False,False,True,False,False,False,False,False
freq,30923,36189,38268,38248,36659,34638,22117,38111,36415,37353,38268


Some columns in some PHEME events have only one unique value for all tweets. Instead of dropping them, we'll just be aware of them because they may vary across PHEME datasets.

In [62]:
for col in gw.columns:
    if len(gw[col].unique()) == 1:
        print("Warning, column `%s` only has one unique value \"%s\"" % (col, gw[col][0]))

Warning, column `event` only has one unique value "charliehebdo"
Warning, column `is_truncated` only has one unique value "0"
Warning, column `user.notifications` only has one unique value "0"
Warning, column `Adverb` only has one unique value "0"


The tweet-level data, we'll use in our data analysis will be in this form.

In [63]:
gw.describe()

,is_rumor,is_source_tweet,tweet_length,symbol_count,user_mentions,urls_count,media_count,hashtags_count,retweet_count,favorite_count,...,sentimentscore,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,has_url_in_text
count,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,...,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.000000,38268.0,38268.000000
mean,0.191936,0.054327,97.651249,0.000026,1.767090,0.115710,0.100711,0.275792,24.313578,11.876450,...,0.025037,5.837044,2.326278,0.927851,0.672468,0.148662,0.223738,0.388027,0.0,0.178713
std,0.393828,0.226666,37.748508,0.005112,1.044231,0.337685,0.301210,0.700195,478.360416,260.287571,...,0.297044,2.877994,1.787683,1.039827,0.967401,0.430173,0.557142,0.716427,0.0,0.383117
min,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.000000,0.000000,67.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.000000,0.000000,107.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,6.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,0.000000,0.000000,133.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.100000,8.000000,3.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.000000
max,1.000000,1.000000,152.000000,1.000000,9.000000,5.000000,2.000000,10.000000,74130.000000,37983.000000,...,1.000000,26.000000,11.000000,8.000000,11.000000,5.000000,11.000000,7.000000,0.0,1.000000


## Thread Level Features

* **Bold features** represent high performing features identified in C. Buntain and J. Golbeck, ["Automatically Identifying Fake News in Popular Twitter Threads"](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8118443&isnumber=8118402)
* Features that are normalized are normalized by thread length


| Name                | Description                               | Type    | Notes |
| ---                 | ---                                       | ---     | ----- |
| thread              | Tweet ID of the source tweet              | `str`   | |
| favorite_count      | Normalized favorite total                 | `float` | |
| retweet_count       | Normlaized retweet total                  | `float` | |
| **hashtags_count**  | Normlaized hashtag total                  | `float` | |
| urls_count          | URL total normalized by thread length     | `float`  | |
| user.tweets_count   | Total tweets by thread users              | `float` | |
| event               | Name of PHEME event                       | `str`  | |
| is_rumor            | Either rumor or nonrumor                  | `bool` | |
| thread_length       | Number of tweets in the thread            | `int`  | |
| user.has_bg_img     | Ratio of users who have bg image          | `float`| |
| user.default_pic    | Ratio of users with default profile pic   | `float`| |
| **has_smile_emoji** | Number of smile emojis in the thread      | `int`  | 😊 is the smile emoji |
| user.verified       | Count of verified users in the thread normalized by thread length     | `float`  | |
| **src.followers_count** | The number of followers of the original poster of the thread. | `int` | |
| src.listed_count    | TODO | `int` | |
| src.user_verified   | TODO | `int` | |
| src.tweets_total    | TODO | `int` | |
| reply_var           | The variance in the timestamps of responses to the source tweet | `float` |
| src_age             | Difference in src user's creation and tweet creation            | `int`   | Measured in seconds |
| time_to_first_resp  | The difference between tweet creation datetime and 1st reply    | `int`   | Measured in seconds |
| time_to_last_resp   | The difference between tweet creation datetime and last reply   | `int`   | Measured in seconds |

In [64]:
def agg_tweets_by_thread(df):
    
    shared = lambda x: 1 - len(set(x)) / len(x)
    shared.__name__ = "shared"

    funcs = [np.mean, sum, np.var]
    agg_props = {
        "favorite_count": funcs,
        "user_mentions": funcs,
        "media_count": funcs,
        "sensitive": funcs,
        "has_place": funcs,
        "has_coords": funcs,
        "retweet_count": funcs,
        "hashtags_count": funcs + [shared],
        "urls_count": funcs,
        "user.tweets_count": funcs,
        "is_rumor": max,
        "tweet_id": len,
        "user.has_bg_img": funcs,
        "has_quest": funcs,
        "has_exclaim": funcs,
        "has_quest_or_exclaim": funcs,
        "user.default_pic": funcs,
        "has_smile_emoji": funcs,
        "user.verified": funcs,
        "user.name_length": funcs,
        "user.handle_length": funcs,
        "user.profile_sbcolor": funcs,
        "user.profile_bgcolor": funcs,
        
        "hasperiod": funcs,
        "number_punct": funcs,
        "negativewordcount" : funcs,
        "positivewordcount" : funcs,
        "capitalratio" : funcs,
        "contentlength" : funcs,
        "sentimentscore" : funcs,
        "Noun" : funcs,
        "Verb" : funcs,
        "Adjective" : funcs,
        "Pronoun" : funcs,
        "Adverb": funcs,
    }
    rename = {
        "tweet_id": "thread_length"
    }

    def g(x):
        # Add size of largest user-to-user conversation component in each thread        
        d = []
        thread_tweets = list(x["tweet_id"])
        G = nx.from_pandas_edgelist(df[df.tweet_id.isin(thread_tweets)], "user_id", "in_reply_user")
        Gc = max(nx.connected_component_subgraphs(G), key=len)
        d.append(nx.number_connected_components(G))
        d.append(nx.diameter(Gc))
        return pd.Series(d, index=["component_count", "largest_cc_diameter"])
    
    # Step 0: Build graph-based features
    graph = df.groupby("thread").apply(g)
    
    # Step 1: Build simple aggregate features
    agg = df.groupby("thread")\
        .agg(agg_props)\
        .rename(columns=rename)
    
    agg.columns = [ "_".join(x) for x in agg.columns.ravel() ]
    agg = agg.rename(columns={"is_rumor_max": "is_rumor", "thread_length_len": "thread_length"})
    
    # Step 2: Builds some features off the source tweet, which has tweet_id == thread            
    src = df[df["is_source_tweet"] == 1][["thread",
                                          "user.followers_count", 
                                          "user.listed_count",
                                          "user.verified",
                                          "created",
                                          "user.created_at",
                                          "user.tweets_count"]] \
                         .rename(columns={"user.followers_count": "src.followers_count",
                                          "user.listed_count": "src.listed_count",
                                          "user.verified": "src.user_verified",
                                          "user.created_at": "src.created_at",
                                          "user.tweets_count": "src.tweets_total"})
    
    # Step 3: Build features off of the reply tweets
    def f(x):
        d = []
        
        # Get various features from the distribution of times of reply tweet
        d.append(min(x["created"]))
        d.append(max(x["created"]))
        d.append(np.var(x["created"]))
                
        return pd.Series(d, index=["first_resp", "last_resp","resp_var"])
        
    replies = df[df["is_source_tweet"] == False] \
        .groupby("thread") \
        .apply(f)

    graph_features = df.groupby("thread").apply(g)
    
    dfs = [agg, src, replies, graph]
    thrd_data = reduce(lambda left, right: pd.merge(left,right, on="thread"), dfs)
    
    # Step 3: Add miscelaneous features
    # Remember timestamps increase as time progresses
    # src.created_at < created < first_resp < last_resp
    thrd_data["time_to_first_resp"] = thrd_data["first_resp"] - thrd_data["created"]
    thrd_data["time_to_last_resp"] = thrd_data["last_resp"] - thrd_data["created"]
    
    return thrd_data

In [ ]:
gw_thrds = agg_tweets_by_thread(gw)
gw_thrds.columns.values

In [ ]:
gw_thrds.head()

In [ ]:
gw_thrds.shape

In [ ]:
fn = "data/threads/%s.csv" % event
gw_thrds.to_csv(fn, index=False)
"Wrote data to %s" % fn